# Topic Modelling for News

![](https://images.unsplash.com/photo-1495020689067-958852a7765e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80)

Photo by [Roman Kraft](https://unsplash.com/photos/_Zua2hyvTBk)

This exercise is about modelling the main topics of a database of News headlines.

Begin by importing the needed libraries:

In [20]:
# TODO: import needed libraries
import nltk
import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models

Load the data in the file `random_headlines.csv`

In [4]:
# TODO: load the dataset
df = pd.read_csv('random_headlines.csv')
print(df.shape)
df.head()

(20000, 2)


,publish_date,headline_text
0,20120305,ute driver hurt in intersection crash
1,20081128,6yo dies in cycling accident
2,20090325,bumper olive harvest expected
3,20100201,replica replaces northernmost sign
4,20080225,woods targets perfect season


This is always a good idea to perform some EDA (exploratory data analytics) on a dataset...

In [5]:
# TODO: Perform a short EDA
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   publish_date   20000 non-null  int64 
 1   headline_text  20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


Now perform all the needed preprocessing on those headlines: case lowering, tokenization, punctuation removal, stopwords removal, stemming/lemmatization.

In [7]:
# TODO: Preprocess the input data
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

#tokenize
df['token'] = df['headline_text'].apply(lambda row: nltk.word_tokenize(row))
                                           
#remove punctuation
df['alpha'] = df['token'].apply(lambda row: [
    word for word in row if word.isalpha()
])

#remove stopwords
stop_words = nltk.corpus.stopwords.words('english')
df['stop'] = df['alpha'].apply(lambda row: [
    word for word in row if word not in stop_words
])

#stemming
stemmer = PorterStemmer()
df['stem'] = df['stop'].apply(lambda row: [
    stemmer.stem(word) for word in row
])

df['stem'].head()

0    [ute, driver, hurt, intersect, crash]
1                       [die, cycl, accid]
2          [bumper, oliv, harvest, expect]
3    [replica, replac, northernmost, sign]
4          [wood, target, perfect, season]
Name: stem, dtype: object

Now use Gensim to compute a BOW

In [8]:
# TODO: Compute the BOW using Gensim
from gensim.corpora import Dictionary
dct = Dictionary(df['stem'])
corpus = [dct.doc2bow(line) for line in df['stem']]
corpus[0:2]

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(5, 1), (6, 1), (7, 1)]]

Compute the TF-IDF using Gensim

In [14]:
# TODO: Compute TF-IDF
from gensim.models import TfidfModel

tfidf_model = TfidfModel(corpus)
tf_idf = tfidf_model[corpus]
print(np.shape(tf_idf))

(20000,)


Finally compute the **LSA** (also called LSI) using Gensim, for a given number of Topics that you choose yourself

In [10]:
# TODO: Compute LSA
from gensim.models import LsiModel
lsa = LsiModel(corpus=corpus, id2word=dct, num_topics=4)

For each of the topic, show the most significant words.

In [11]:
# TODO: Print the 3 or 4 most significant words of each topic
lsa.print_topics(num_words=3)

[(0, '-0.752*"polic" + -0.404*"man" + -0.207*"charg"'),
 (1, '0.670*"man" + -0.574*"polic" + 0.329*"charg"'),
 (2, '-0.655*"new" + -0.296*"plan" + -0.242*"say"'),
 (3, '0.703*"new" + -0.343*"say" + -0.334*"plan"')]

What do you think about those results?

In [ ]:
police,man,charge,new,plan,say are the frequently used words in news articles

Now let's try to use LDA instead of LSA using Gensim

In [15]:
# TODO: Compute LDA
from gensim.models import LdaModel
lda = LdaModel(corpus=corpus, id2word=dct, num_topics=4, random_state=0, chunksize=512, passes=5)

In [16]:
# TODO: print the most frequent words of each topic
lda.print_topics(num_words=3)

[(0, '0.016*"report" + 0.009*"back" + 0.009*"may"'),
 (1, '0.012*"mine" + 0.011*"polic" + 0.009*"elect"'),
 (2, '0.013*"question" + 0.010*"council" + 0.010*"fund"'),
 (3, '0.012*"sydney" + 0.012*"charg" + 0.011*"australian"')]

Now, how does it work with LDA?

In [ ]:
report, back,may,mine,police,elect,council, question,fund,sydney,charge,australian

In [18]:
!pip install pyldavis --user

In [22]:
conda install -c conda-forge pyldavis

Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda




Let's make some visualization of the LDA results using pyLDAvis.

Depending on your results, you can try to fine tune the algorithm: number of topics, hyperparameters...
And check with others their results.

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
v = pyLDAvis.gensim_models.prepare(lda, corpus, dct)
v
